# Getting Started with CLIMADA

This notebook provides an interactive introduction to CLIMADA for climate risk analysis.

## What is CLIMADA?

CLIMADA (CLIMate ADAptation) is an open-source platform for probabilistic natural catastrophe impact modeling. It combines:

- **Hazard** data (storms, floods, droughts, etc.)
- **Exposure** data (assets, population, infrastructure)
- **Vulnerability** functions (impact models)

To calculate **Risk** = Hazard × Exposure × Vulnerability

## Setup

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Import CLIMADA modules
from climada.entity import Exposures, ImpactFunc, ImpactFuncSet
from climada.hazard import TropCyclone, Hazard
from climada.engine import Impact

# Check CLIMADA version
import climada
print(f"CLIMADA version: {climada.__version__}")

## 1. Working with Exposure Data

Exposure data represents what is at risk: buildings, infrastructure, population, or economic assets.

In [ ]:
# Create a simple exposure dataset
exposure = Exposures()

# Define asset locations and values
exposure.gdf['latitude'] = np.array([26.0, 26.5, 27.0, 27.5, 28.0])
exposure.gdf['longitude'] = np.array([-80.0, -80.2, -80.1, -79.9, -80.0])
exposure.gdf['value'] = np.array([1e6, 2e6, 1.5e6, 3e6, 2.5e6])  # USD

# Set metadata
exposure.ref_year = 2024
exposure.value_unit = 'USD'

# Validate
exposure.check()

print(f"Total exposure value: ${exposure.gdf['value'].sum():,.0f}")
exposure.gdf

In [ ]:
# Visualize exposure
fig, ax = plt.subplots(figsize=(10, 6))

scatter = ax.scatter(
    exposure.gdf['longitude'],
    exposure.gdf['latitude'],
    s=exposure.gdf['value'] / 1e4,
    c=exposure.gdf['value'],
    cmap='YlOrRd',
    alpha=0.6,
    edgecolors='black'
)

ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Exposure Distribution')
plt.colorbar(scatter, ax=ax, label='Asset Value (USD)')
ax.grid(True, alpha=0.3)
plt.show()

## 2. Defining Impact Functions

Impact functions (vulnerability curves) define how exposure responds to hazard intensity.

In [ ]:
# Create a tropical cyclone impact function
impf = ImpactFunc()
impf.id = 1
impf.name = 'TC Building Damage'
impf.haz_type = 'TC'
impf.intensity_unit = 'm/s'

# Define wind speed thresholds and damage
impf.intensity = np.array([0, 20, 30, 40, 50, 60, 70, 80])
impf.mdd = np.array([0, 0.01, 0.05, 0.15, 0.35, 0.60, 0.85, 1.0])  # Mean Damage Degree
impf.paa = np.array([0, 0.5, 0.7, 0.85, 0.95, 0.98, 1.0, 1.0])    # % Affected Assets

# Validate
impf.check()

print(f"Impact function created: {impf.name}")

In [ ]:
# Visualize impact function
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(impf.intensity, impf.mdd, 'o-', label='Mean Damage Degree (MDD)', linewidth=2)
ax.plot(impf.intensity, impf.paa, 's--', label='% Affected Assets (PAA)', linewidth=2)

ax.set_xlabel('Wind Speed (m/s)')
ax.set_ylabel('Damage Ratio / Affected Fraction')
ax.set_title('Tropical Cyclone Impact Function')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(-0.05, 1.05)
plt.show()

## 3. Understanding Hazard Data

Hazard data describes probabilistic event sets with intensities and frequencies.

**Note:** Loading real hazard data requires internet connection and may take time on first use.

In [ ]:
# Example: Create a simple hazard structure
# In practice, use TropCyclone.from_ibtracs() for real data

print("To load real tropical cyclone data from IBTrACS:")
print("""\nhazard = TropCyclone.from_ibtracs_netcdf(
    provider='official',
    basin='NA',  # North Atlantic
    year_range=(2017, 2018)
)
""")

print("\nNote: This downloads data on first use and may take several minutes.")
print("Uncomment the code below to try it:")

# Uncomment to load real data:
# hazard = TropCyclone.from_ibtracs_netcdf(
#     provider='official',
#     basin='NA',
#     year_range=(2017, 2018)
# )
# print(f"Loaded {hazard.size} storm events")

## 4. Calculating Risk

With exposure, hazard, and impact functions, we can calculate risk (expected impacts).

In [ ]:
# Create impact function set
impf_set = ImpactFuncSet()
impf_set.append(impf)

print("Risk calculation: Impact = Hazard × Exposure × Vulnerability")
print("\nTo calculate impacts with real data:")
print("""\nimpact = Impact()
impact.calc(exposure, hazard, impf_set)

# Results
print(f"Average Annual Impact: ${impact.aai_agg:,.0f}")
impact.plot_exceedance()  # Return period curve
""")

print("\nNote: Requires complete hazard data to calculate impacts.")

## 5. Using LitPop for Economic Exposure

CLIMADA's LitPop module generates exposure from nightlight and GDP data.

In [ ]:
from climada.entity.exposures import LitPop

print("To generate LitPop exposure for a country:")
print("""\n# Example: Switzerland
exposure_litpop = LitPop.from_countries(
    countries=['CHE'],  # ISO 3166 alpha-3 code
    res_arcsec=300,     # Resolution (~10km)
    reference_year=2020
)

print(f"Generated {len(exposure_litpop.gdf)} exposure points")
print(f"Total value: ${exposure_litpop.gdf['value'].sum()/1e9:.2f} billion")
""")

print("\nNote: Downloads data on first use. Uncomment above to try.")

## 6. Scenario Analysis

Compare current vs. future climate scenarios.

In [ ]:
# Example scenario comparison
scenarios = {
    'Baseline 2024': {'year': 2024, 'climate': 'historical', 'exposure_growth': 1.0},
    'Future 2050 (RCP4.5)': {'year': 2050, 'climate': 'RCP4.5', 'exposure_growth': 1.5},
    'Future 2050 (RCP8.5)': {'year': 2050, 'climate': 'RCP8.5', 'exposure_growth': 1.5},
}

# Visualize scenario differences
fig, ax = plt.subplots(figsize=(12, 6))

scenario_names = list(scenarios.keys())
# Simulated risk values (in practice, calculate from Impact objects)
risk_values = [100, 150, 200]  # Million USD

bars = ax.bar(scenario_names, risk_values, color=['green', 'yellow', 'red'], alpha=0.7)
ax.set_ylabel('Average Annual Loss (Million USD)')
ax.set_title('Risk Comparison Across Climate Scenarios')
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, val in zip(bars, risk_values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'${val}M', ha='center', va='bottom')

plt.xticks(rotation=15, ha='right')
plt.tight_layout()
plt.show()

## Next Steps

1. **Explore Example Workflows**: Check the `workflows/` directory for detailed scripts
   - `01_basic_risk_assessment.py`
   - `02_exposure_integration.py`
   - `03_hazard_integration.py`
   - `04_vulnerability_functions.py`
   - `05_scenario_development.py`

2. **Use Real Data**:
   - Load historical hazards from IBTrACS
   - Generate exposure with LitPop
   - Download climate projections

3. **Learn More**:
   - [CLIMADA Documentation](https://climada-python.readthedocs.io/)
   - [CLIMADA Tutorials](https://climada-python.readthedocs.io/en/stable/user-guide/)
   - [GitHub Repository](https://github.com/CLIMADA-project/climada_python)

4. **Advanced Topics**:
   - Calibrate impact functions with observed data
   - Assess cost-benefit of adaptation measures
   - Create custom hazard modules
   - Integrate with climate model outputs